This script gathers perpetual future snapshots from kaiko

In [1]:
import requests
import pandas as pd
import datetime
import time

In [2]:
def process_kaiko(raw):
#trading data
    for i in range(len(raw)):
                    epoch = int(raw[i]['poll_timestamp']/1000)
                    time = datetime.datetime.utcfromtimestamp(epoch)
                
                    mid_price = raw[i]['mid_price']
                    spread = raw[i]['spread']
                    ask_slippage = raw[i]['ask_slippage']
                    bid_slippage = raw[i]['bid_slippage']
                    asks = raw[i]['asks']
                    bids = raw[i]['bids']

                    lst.append([epoch, time, mid_price, spread, ask_slippage, bid_slippage, asks, bids])
    return

In [3]:
symbol = 'eth-usdt'
slippage = 1e6
lst = []

oderbook = 'snapshots'
#oderbook = 'ob_aggregations'
interval = '1h'

#end_time = datetime.datetime.utcnow()
#start_time = end_time - datetime.timedelta(minutes=50)
#end_time_iso = end_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

start_time = datetime.datetime.utcfromtimestamp(1661126400) #22.08.2022
start_time_iso = start_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

In [4]:
counter = 0
cont = True
while cont:
    counter += 1
    header = {'X-Api-Key': '68cf96d421857181898149886cc12ddc'}
    query = f'https://eu.market-api.kaiko.io/v2/data/order_book_snapshots.v1/exchanges/binc/perpetual-future/{symbol}/{oderbook}/full?start_time={start_time_iso}&slippage={slippage}&sort=asc&page_size=100&slippage_ref=best'
    response = requests.get(query, headers=header)
    raw = response.json()
    next_url = raw['next_url']
    raw = raw['data']
    
    print(counter, ': ', start_time_iso, 'to', raw[-1]['poll_date'])
    process_kaiko(raw)
  
    start_time_iso = raw[-1]['poll_date']
    
    #end_time_iso = start_time_iso
    #start_time = start_time - datetime.timedelta(minutes=50)
    #start_time_iso = start_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    
    time.sleep(2)
    
    if int(raw[-1]['poll_timestamp']/1000) >= 1662987600: #today
        cont = False

1 :  2022-08-22T00:00:00.000000Z to 2022-08-22T00:41:26.094Z
2 :  2022-08-22T00:41:26.094Z to 2022-08-22T01:22:41.135Z
3 :  2022-08-22T01:22:41.135Z to 2022-08-22T02:03:56.111Z
4 :  2022-08-22T02:03:56.111Z to 2022-08-22T02:45:10.991Z
5 :  2022-08-22T02:45:10.991Z to 2022-08-22T03:26:26.260Z
6 :  2022-08-22T03:26:26.260Z to 2022-08-22T04:07:40.942Z
7 :  2022-08-22T04:07:40.942Z to 2022-08-22T04:48:55.928Z
8 :  2022-08-22T04:48:55.928Z to 2022-08-22T05:30:10.991Z
9 :  2022-08-22T05:30:10.991Z to 2022-08-22T06:11:25.995Z
10 :  2022-08-22T06:11:25.995Z to 2022-08-22T06:52:40.958Z
11 :  2022-08-22T06:52:40.958Z to 2022-08-22T07:33:55.957Z
12 :  2022-08-22T07:33:55.957Z to 2022-08-22T08:15:10.953Z
13 :  2022-08-22T08:15:10.953Z to 2022-08-22T08:56:25.963Z
14 :  2022-08-22T08:56:25.963Z to 2022-08-22T09:37:40.954Z
15 :  2022-08-22T09:37:40.954Z to 2022-08-22T10:18:55.944Z
16 :  2022-08-22T10:18:55.944Z to 2022-08-22T11:00:11.868Z
17 :  2022-08-22T11:00:11.868Z to 2022-08-22T11:41:25.941Z
18 

140 :  2022-08-25T23:33:55.949Z to 2022-08-26T00:15:11.131Z
141 :  2022-08-26T00:15:11.131Z to 2022-08-26T00:56:25.963Z
142 :  2022-08-26T00:56:25.963Z to 2022-08-26T01:37:40.983Z
143 :  2022-08-26T01:37:40.983Z to 2022-08-26T02:18:56.237Z
144 :  2022-08-26T02:18:56.237Z to 2022-08-26T03:00:11.904Z
145 :  2022-08-26T03:00:11.904Z to 2022-08-26T03:41:26.141Z
146 :  2022-08-26T03:41:26.141Z to 2022-08-26T04:22:40.984Z
147 :  2022-08-26T04:22:40.984Z to 2022-08-26T05:03:55.925Z
148 :  2022-08-26T05:03:55.925Z to 2022-08-26T05:45:10.921Z
149 :  2022-08-26T05:45:10.921Z to 2022-08-26T06:26:26.022Z
150 :  2022-08-26T06:26:26.022Z to 2022-08-26T07:07:40.948Z
151 :  2022-08-26T07:07:40.948Z to 2022-08-26T07:48:55.990Z
152 :  2022-08-26T07:48:55.990Z to 2022-08-26T08:30:11.038Z
153 :  2022-08-26T08:30:11.038Z to 2022-08-26T09:11:26.030Z
154 :  2022-08-26T09:11:26.030Z to 2022-08-26T09:52:41.025Z
155 :  2022-08-26T09:52:41.025Z to 2022-08-26T10:33:56.055Z
156 :  2022-08-26T10:33:56.055Z to 2022-

277 :  2022-08-29T21:45:36.486Z to 2022-08-29T22:26:51.029Z
278 :  2022-08-29T22:26:51.029Z to 2022-08-29T23:08:05.981Z
279 :  2022-08-29T23:08:05.981Z to 2022-08-29T23:49:21.137Z
280 :  2022-08-29T23:49:21.137Z to 2022-08-30T00:30:36.013Z
281 :  2022-08-30T00:30:36.013Z to 2022-08-30T01:11:51.440Z
282 :  2022-08-30T01:11:51.440Z to 2022-08-30T01:53:05.929Z
283 :  2022-08-30T01:53:05.929Z to 2022-08-30T02:34:21.114Z
284 :  2022-08-30T02:34:21.114Z to 2022-08-30T03:15:36.027Z
285 :  2022-08-30T03:15:36.027Z to 2022-08-30T03:56:51.018Z
286 :  2022-08-30T03:56:51.018Z to 2022-08-30T04:38:06.004Z
287 :  2022-08-30T04:38:06.004Z to 2022-08-30T05:19:21.018Z
288 :  2022-08-30T05:19:21.018Z to 2022-08-30T06:00:35.972Z
289 :  2022-08-30T06:00:35.972Z to 2022-08-30T06:41:51.212Z
290 :  2022-08-30T06:41:51.212Z to 2022-08-30T07:23:06.525Z
291 :  2022-08-30T07:23:06.525Z to 2022-08-30T08:04:20.941Z
292 :  2022-08-30T08:04:20.941Z to 2022-08-30T08:45:35.996Z
293 :  2022-08-30T08:45:35.996Z to 2022-

414 :  2022-09-02T19:57:12.293Z to 2022-09-02T20:38:27.434Z
415 :  2022-09-02T20:38:27.434Z to 2022-09-02T21:19:42.159Z
416 :  2022-09-02T21:19:42.159Z to 2022-09-02T22:00:57.156Z
417 :  2022-09-02T22:00:57.156Z to 2022-09-02T22:42:12.511Z
418 :  2022-09-02T22:42:12.511Z to 2022-09-02T23:23:27.278Z
419 :  2022-09-02T23:23:27.278Z to 2022-09-03T00:04:42.250Z
420 :  2022-09-03T00:04:42.250Z to 2022-09-03T00:45:57.430Z
421 :  2022-09-03T00:45:57.430Z to 2022-09-03T01:27:12.163Z
422 :  2022-09-03T01:27:12.163Z to 2022-09-03T02:08:27.256Z
423 :  2022-09-03T02:08:27.256Z to 2022-09-03T02:49:42.151Z
424 :  2022-09-03T02:49:42.151Z to 2022-09-03T03:30:57.144Z
425 :  2022-09-03T03:30:57.144Z to 2022-09-03T04:12:14.448Z
426 :  2022-09-03T04:12:14.448Z to 2022-09-03T04:53:27.172Z
427 :  2022-09-03T04:53:27.172Z to 2022-09-03T05:34:42.187Z
428 :  2022-09-03T05:34:42.187Z to 2022-09-03T06:15:57.167Z
429 :  2022-09-03T06:15:57.167Z to 2022-09-03T06:57:12.170Z
430 :  2022-09-03T06:57:12.170Z to 2022-

551 :  2022-09-06T18:08:27.803Z to 2022-09-06T18:49:44.245Z
552 :  2022-09-06T18:49:44.245Z to 2022-09-06T19:30:57.340Z
553 :  2022-09-06T19:30:57.340Z to 2022-09-06T20:12:12.171Z
554 :  2022-09-06T20:12:12.171Z to 2022-09-06T20:53:27.217Z
555 :  2022-09-06T20:53:27.217Z to 2022-09-06T21:34:43.442Z
556 :  2022-09-06T21:34:43.442Z to 2022-09-06T22:15:58.994Z
557 :  2022-09-06T22:15:58.994Z to 2022-09-06T22:57:12.151Z
558 :  2022-09-06T22:57:12.151Z to 2022-09-06T23:38:27.263Z
559 :  2022-09-06T23:38:27.263Z to 2022-09-07T00:19:44.701Z
560 :  2022-09-07T00:19:44.701Z to 2022-09-07T01:00:57.170Z
561 :  2022-09-07T01:00:57.170Z to 2022-09-07T01:42:12.322Z
562 :  2022-09-07T01:42:12.322Z to 2022-09-07T02:23:27.315Z
563 :  2022-09-07T02:23:27.315Z to 2022-09-07T03:04:43.239Z
564 :  2022-09-07T03:04:43.239Z to 2022-09-07T03:45:57.208Z
565 :  2022-09-07T03:45:57.208Z to 2022-09-07T04:27:12.238Z
566 :  2022-09-07T04:27:12.238Z to 2022-09-07T05:08:27.246Z
567 :  2022-09-07T05:08:27.246Z to 2022-

688 :  2022-09-10T16:19:42.219Z to 2022-09-10T17:00:57.157Z
689 :  2022-09-10T17:00:57.157Z to 2022-09-10T17:42:13.348Z
690 :  2022-09-10T17:42:13.348Z to 2022-09-10T18:23:27.192Z
691 :  2022-09-10T18:23:27.192Z to 2022-09-10T19:04:42.171Z
692 :  2022-09-10T19:04:42.171Z to 2022-09-10T19:45:57.230Z
693 :  2022-09-10T19:45:57.230Z to 2022-09-10T20:27:12.138Z
694 :  2022-09-10T20:27:12.138Z to 2022-09-10T21:08:27.222Z
695 :  2022-09-10T21:08:27.222Z to 2022-09-10T21:49:44.351Z
696 :  2022-09-10T21:49:44.351Z to 2022-09-10T22:30:57.152Z
697 :  2022-09-10T22:30:57.152Z to 2022-09-10T23:12:12.221Z
698 :  2022-09-10T23:12:12.221Z to 2022-09-10T23:53:27.178Z
699 :  2022-09-10T23:53:27.178Z to 2022-09-11T00:34:42.313Z
700 :  2022-09-11T00:34:42.313Z to 2022-09-11T01:15:57.324Z
701 :  2022-09-11T01:15:57.324Z to 2022-09-11T01:57:12.423Z
702 :  2022-09-11T01:57:12.423Z to 2022-09-11T02:38:27.277Z
703 :  2022-09-11T02:38:27.277Z to 2022-09-11T03:19:42.266Z
704 :  2022-09-11T03:19:42.266Z to 2022-

In [5]:
df = pd.DataFrame(lst, columns=['epoch', 'date', 'mid_price', 'spread', 'ask_slippage', 'bid_slippage', 'asks', 'bids'])
df.sort_values(by=['epoch'], ascending=True, inplace=True)
df = df.drop_duplicates(subset=['epoch'], keep='first')
df = df.reset_index(drop=True)
df

,epoch,date,mid_price,spread,ask_slippage,bid_slippage,asks,bids
0,1661126410,2022-08-22 00:00:10,1617.165,0.009999999999990905,0.00038399049544896876,0.00019405585916119195,"[{'amount': '0.944', 'price': '1617.17'}, {'am...","[{'amount': '35.335', 'price': '1617.16'}, {'a..."
1,1661126436,2022-08-22 00:00:36,1616.6149999999998,0.009999999999990905,0.0002102224545043017,0.0002706436762845114,"[{'amount': '36.794', 'price': '1616.62'}, {'a...","[{'amount': '6.851', 'price': '1616.61'}, {'am..."
2,1661126461,2022-08-22 00:01:01,1617.725,0.009999999999990905,0.0003778199898375263,0.00031892302895593057,"[{'amount': '27.845', 'price': '1617.73'}, {'a...","[{'amount': '23.764', 'price': '1617.72'}, {'a..."
3,1661126485,2022-08-22 00:01:25,1617.7350000000001,0.009999999999990905,0.0004120917314521417,0.00017311743973492667,"[{'amount': '1.066', 'price': '1617.74'}, {'am...","[{'amount': '59.891', 'price': '1617.73'}, {'a..."
4,1661126510,2022-08-22 00:01:50,1616.0349999999999,0.009999999999990905,0.0002846212038376322,0.0001012198263728879,"[{'amount': '26.754', 'price': '1616.04'}, {'a...","[{'amount': '20.843', 'price': '1616.03'}, {'a..."
...,...,...,...,...,...,...,...,...
74444,1662987558,2022-09-12 12:59:18,1745.0149999999999,0.009999999999990905,0.00023816250880864825,0.0002541906953371844,"[{'amount': '27.977', 'price': '1745.02'}, {'a...","[{'amount': '69.671', 'price': '1745.01'}, {'a..."
74445,1662987583,2022-09-12 12:59:43,1744.9450000000002,0.009999999999990905,0.00027724788987554207,0.00024543714869095017,"[{'amount': '2.328', 'price': '1744.95'}, {'am...","[{'amount': '7.767', 'price': '1744.94'}, {'am..."
74446,1662987607,2022-09-12 13:00:07,1744.7649999999999,0.009999999999990905,0.00017074425792993177,0.0002854210479829703,"[{'amount': '139.256', 'price': '1744.77'}, {'...","[{'amount': '17.216', 'price': '1744.76'}, {'a..."
74447,1662987632,2022-09-12 13:00:32,1745.295,0.009999999999990905,0.00029200410374670004,0.00013091137751440515,"[{'amount': '16.194', 'price': '1745.3'}, {'am...","[{'amount': '148.891', 'price': '1745.29'}, {'..."


In [6]:
df['ask_slippage'] = df['ask_slippage'].astype('float')
df['bid_slippage'] = df['bid_slippage'].astype('float')
df['spread'] = df['spread'].astype('float')

In [7]:
#store in csv
df.to_csv(f'data/orderbook_eth_usdt.csv', index=False)